In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from datetime import datetime
import os
import geopandas as gpd
import keras
import gee
import utils


In [4]:
#region_name = 'pang_war_myanmar'
region_name = 'myanmar_mines_2'
#region_name = 'test_region_small'

sensor='NICFI'
tile_size = 576 # this is the around the max size that GEE exports allow with 12-band imagery
tile_padding = 48
patchsize=48

if sensor=='NICFI':
    resolution = 4.77
else:
    resolution = 10

start_date = datetime(2024, 3, 1)
end_date = datetime(2024, 7, 1)
clear_threshold = 0.6


region = gpd.read_file(f'../data/boundaries/{region_name}.geojson').geometry[0].__geo_interface__
tiles = utils.create_tiles(region, tile_size, tile_padding, resolution)
print(f"Created {len(tiles):,} tiles")

#model_name = '48px_v4.0_ensemble_2024-06-14'
#model_name = '24px_v4.0_ensemble_2024-06-17'
#model_name = '48px_v4.1_ensemble_2024-06-28' # nicfi 
model_name = f'{sensor}_{patchsize}px_v4.1_ensemble_2024-07-12'

model = keras.models.load_model(f'../models/{model_name}.h5', compile=False)

Created 64 tiles


2024-07-12 13:00:31.836215: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-07-12 13:00:31.836255: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-07-12 13:00:31.836304: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-07-12 13:00:31.836361: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-12 13:00:31.836387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
import gee

pred_threshold = 0.65

data_pipeline = gee.Data_Extractor(
    tiles, 
    start_date, 
    end_date,  
    clear_threshold, 
    10,
    sensor
    )
#data_pipeline.composite
predictions=data_pipeline.make_predictions(model, pred_threshold=pred_threshold, tries=5)


 14%|█▍        | 1/7 [00:15<01:31, 15.30s/it]

Found 2 positives.


 29%|██▊       | 2/7 [00:30<01:15, 15.05s/it]

Found 3 positives.


 43%|████▎     | 3/7 [00:54<01:16, 19.18s/it]

...

 86%|████████▌ | 6/7 [01:46<00:18, 18.17s/it]

Found 17 positives.


100%|██████████| 7/7 [01:58<00:00, 16.99s/it]


Found 17 positives.


0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [6]:
import plotly.express as px
import geemap
import ee

predictions['lat'] = predictions['geometry'].apply(lambda x: x.centroid.y)
predictions['lon'] = predictions['geometry'].apply(lambda x: x.centroid.x)

#convert predictions to feature collection

gdf = gpd.GeoDataFrame(predictions, geometry=gpd.points_from_xy(predictions.lon, predictions.lat))
fc = geemap.geopandas_to_ee(predictions)
center=[predictions['lat'].mean(), predictions['lon'].mean()]
#satellite basemap
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/asia')
nicfi=nicfi.filterDate(start_date, end_date).median()
nicfi=nicfi.visualize(min=0, max=2000, bands=['R', 'G', 'B'])

Map = geemap.Map(center=center, zoom=10, add_google_map=False)
Map.addLayer(nicfi, {}, 'NICFI')
Map.addLayer(fc, {'color':'red'}, 'Predictions')

Map

Map(center=[21.43527959845305, 100.41561978170054], controls=(WidgetControl(options=['position', 'transparent_…

In [96]:
# write the predictions to a file
print(len(predictions), 'chips with predictions above', pred_threshold)
# write the predictions to a file
model_version_name = '_'.join(model_name.split('_')[0:2])
# if the outputs directory does not exist, create it
if not os.path.exists(f'../data/outputs/{model_version_name}'):
    os.makedirs(f'../data/outputs/{model_version_name}')
time_period = f"{start_date.month}_{start_date.year}-{end_date.month}_{end_date.year}"
predictions.to_file(f"../data/outputs/{model_version_name}/{region_name}_{model_version_name}_{pred_threshold:.2f}_{time_period}.geojson", driver="GeoJSON")

1677 chips with predictions above 0.85
